In [ ]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
import os
from tensorflow.python.keras import layers, Sequential,losses, metrics, optimizers, callbacks, models
from tensorflow.python.keras.layers import Dense, GlobalAveragePooling2D, Dropout
from tensorflow.python.keras.models import Model
from tensorflow.python.keras.applications import vgg16, resnet
from tensorflow.python.keras.optimizer_v2 import adam

In [ ]:
image_height = 48
image_width = 48
emotions_count = 8
emotion_labels = ['neutral', 'happiness', 'surprise', 'sadness', 'anger', 'disgust', 'fear', 'contempt']

samples = 35393           # 2~35394
training_samples = 28317  # 2~28318 (Training)
validation_samples = 3541 # 28319~31859 (PublicTest)
test_samples = 3535       # 31860~35394 (PrivateTest)

In [ ]:
# load data
image_path = "./dataset/images.npy"
emotion_path = "./dataset/emotions_multi.npy"

images = np.load(image_path)
emotions = np.load(emotion_path)

print(images.shape)
print(emotions.shape)

In [ ]:
tf.config.run_functions_eagerly(True)
def model_acc(y_true, y_pred):
    size = y_true.shape[0]
    acc = 0
    for i in range(size):
        true = y_true[i]
        pred = y_pred[i]           
        index_max = tf.argmax(pred).numpy()
        if true[index_max].numpy()==tf.reduce_max(true).numpy():
            acc += 1
    return acc/size

In [ ]:
def train(model, learning_rate, loss, num_epochs, batch_size):
    model.compile(optimizer=adam.Adam(learning_rate=learning_rate), 
                  loss=losses.CategoricalCrossentropy(), 
                  metrics = [model_acc])
    return model.fit(x=training_images,
                     y=training_emotions,
                     batch_size=batch_size,
                     epochs=num_epochs,
                     validation_data=(test_images, test_emotions))

In [ ]:
def create_model():
    base_model = vgg16.VGG16(include_top=False, 
                             weights="imagenet", 
                             input_shape=(48,48,3))
    base_model.trainable=True
    return Sequential([
        base_model,
        layers.GlobalAveragePooling2D(),
        layers.Dense(4096, activation='relu'),
        Dropout(0.5),
        layers.Dense(4096, activation='relu'),
        Dropout(0.5),
        layers.Dense(emotions_count, activation='softmax'),
    ])

In [ ]:
images = tf.convert_to_tensor(images)
emotions = tf.convert_to_tensor(emotions)
training_size = training_samples + validation_samples
test_size = test_samples

training_emotions = emotions[:training_size]
test_emotions = emotions[training_size:]
print("training_emotions shape:", training_emotions.shape)
print("test_emotions shape:", test_emotions.shape)

In [ ]:
val_accs = []
for i in range(3):
    if i == 0: # 0~255
        images_rescaled = tf.image.grayscale_to_rgb(images)
    if 1 == 1: # -1~1
        images_rescaled = tf.image.grayscale_to_rgb(layers.Rescaling(1./127.5, offset= -1)(images))
    if i == 2: # 0~1
        images_rescaled = tf.image.grayscale_to_rgb(layers.Rescaling(1./255.0, offset= 0)(images))
    training_images = images_rescaled[:training_size]
    test_images = images_rescaled[training_size:]
    
    print("training_images shape:", training_images.shape)
    print("test_images shape:", test_images.shape)

    model = create_model()
    learning_rate = 1e-4
    num_epochs = 40
    loss = losses.MeanSquaredError()
    batch_size = 32
    history1 = train(model, learning_rate , loss, num_epochs, batch_size)
    val_accs.append(history1['val_acc'])

In [ ]:
# model comparison
val_acc1 = val_accs[0]
val_acc2 = val_accs[1]
val_acc3 = val_accs[2]
epochs = range(num_epochs)
print(val_acc1)
print(val_acc2)
print(val_acc3)

In [ ]:
fig = plt.figure()
plt.plot(epochs, val_acc1, label = "line 1")
plt.plot(epochs, val_acc2, label = "line 2")
plt.plot(epochs, val_acc3, label = "line 3")
plt.legend()
plt.show()

In [ ]:
# save plot
save_plot_file_name = "exp1.png"
if not os.path.isdir('./results/'):
    os.mkdir('./results/')
fig.savefig(os.path.join('./results/', save_plot_file_name))